In [ ]:
pip install grad-cam opencv-python torch torchvision matplotlib


In [ ]:
import sys, os
import pandas as pd
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision.models as models

# Add src folder to path
PROJECT_ROOT = os.path.abspath("..")
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from extract_image_features import extract  # you already have this file


In [ ]:
# Load trained CNN model used for embeddings
device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

resnet = models.resnet18(pretrained=True)
resnet.fc = torch.nn.Identity()
resnet.to(device)
resnet.eval()


In [ ]:
# Grad-CAM setup
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

def run_gradcam(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print("Could not load:", image_path)
        return None

    img = cv2.resize(img, (224, 224))
    img_rgb = img[..., ::-1]  # BGR → RGB
    img_rgb = img_rgb.astype(np.float32) / 255.0  # normalize for CAM overlay

    input_tensor = transform(img_rgb).unsqueeze(0).to(device)

    cam = GradCAM(model=resnet, target_layers=[resnet.layer4[-1]])
    grayscale_cam = cam(input_tensor=input_tensor)[0]

    cam_overlay = show_cam_on_image(img_rgb, grayscale_cam)
    return cam_overlay


In [ ]:
import os
train = pd.read_excel("../data/raw/train.xlsx")
def attach_images(df):
    df = df.copy()
    df["image_path"] = df["id"].astype(str).apply(
        lambda x: "../data/images/all/" + x + ".png"
    )
    return df

train = attach_images(train)

# optional check
print("Example paths:", train["image_path"].head(3).tolist())
print("Total rows:", len(train))
print("Images exist:", train["image_path"].apply(os.path.exists).sum())


In [ ]:
sample_paths = train["image_path"].dropna().sample(5, random_state=42).tolist()

for path in sample_paths:
    print("\nExplaining:", path)
    cam_img = run_gradcam(path)
    if cam_img is not None:
        plt.imshow(cam_img)
        plt.axis("off")
        plt.show()
